In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor

In [3]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League','Division','NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary','League','Division','NewLeague'], axis = 1).astype("float64")
X = pd.concat([X_,dms[['League_N','Division_W','NewLeague_N']]], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [1]:
#!pip install catboost

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=15daa9bd468bf4873bfe64045c8a8d75b5bd077e14cc524363e98f5d36a6ee80
  Stored in directory: c:\users\tolga\appdata\local\pip\cache\wheels\f9\8d\8d\f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [5]:
catb = CatBoostRegressor()
catb_model = catb.fit(X_train, y_train)

Learning rate set to 0.029229
0:	learn: 438.1974206	total: 161ms	remaining: 2m 40s
1:	learn: 432.4168868	total: 164ms	remaining: 1m 21s
2:	learn: 426.3836690	total: 169ms	remaining: 56s
3:	learn: 420.2261014	total: 172ms	remaining: 42.9s
4:	learn: 414.9976675	total: 176ms	remaining: 35s
5:	learn: 409.6125323	total: 180ms	remaining: 29.8s
6:	learn: 403.9277911	total: 183ms	remaining: 25.9s
7:	learn: 398.4395285	total: 186ms	remaining: 23.1s
8:	learn: 392.4517081	total: 188ms	remaining: 20.7s
9:	learn: 387.4871123	total: 190ms	remaining: 18.8s
10:	learn: 382.6230510	total: 191ms	remaining: 17.2s
11:	learn: 378.1012454	total: 193ms	remaining: 15.8s
12:	learn: 372.6002306	total: 194ms	remaining: 14.7s
13:	learn: 368.4682192	total: 196ms	remaining: 13.8s
14:	learn: 364.0565766	total: 197ms	remaining: 12.9s
15:	learn: 359.5683249	total: 199ms	remaining: 12.2s
16:	learn: 355.1782794	total: 200ms	remaining: 11.6s
17:	learn: 350.4689946	total: 202ms	remaining: 11s
18:	learn: 346.2553348	total: 

In [ ]:
#TAHMİN

In [6]:
y_pred = catb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

350.2683163098795

In [ ]:
#TUNING

In [10]:
catb_params = {
    'iterations':[200,500,1000,2000],
    'learning_rate':[0.01,0.03,0.05,0.1],
    'depth':[3,4,5,6,7,8]
}

In [11]:
catb_model = CatBoostRegressor()
catb_cv_model = GridSearchCV(catb_model, catb_params, cv = 5, n_jobs = -1, verbose = 2)

In [12]:
catb_cv_model.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  8.7min finished


0:	learn: 422.4143448	total: 29.9ms	remaining: 29.9s
1:	learn: 404.1864276	total: 31.1ms	remaining: 15.5s
2:	learn: 386.3231718	total: 32.1ms	remaining: 10.7s
3:	learn: 370.5548032	total: 33ms	remaining: 8.23s
4:	learn: 354.9242038	total: 34.1ms	remaining: 6.79s
5:	learn: 342.3403984	total: 35.2ms	remaining: 5.83s
6:	learn: 328.2370070	total: 36.2ms	remaining: 5.14s
7:	learn: 317.5056526	total: 37.6ms	remaining: 4.66s
8:	learn: 306.6243511	total: 39ms	remaining: 4.29s
9:	learn: 297.3147023	total: 40.8ms	remaining: 4.04s
10:	learn: 288.3685892	total: 43.4ms	remaining: 3.9s
11:	learn: 281.0996220	total: 44.8ms	remaining: 3.69s
12:	learn: 273.2254898	total: 46.4ms	remaining: 3.52s
13:	learn: 266.9003385	total: 47.8ms	remaining: 3.37s
14:	learn: 261.9092500	total: 49.2ms	remaining: 3.23s
15:	learn: 256.2637350	total: 50.5ms	remaining: 3.11s
16:	learn: 250.3667935	total: 51.6ms	remaining: 2.98s
17:	learn: 244.8631098	total: 52.8ms	remaining: 2.88s
18:	learn: 240.1540669	total: 54.5ms	remain

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x0000021D131C0208>,
             n_jobs=-1,
             param_grid={'depth': [3, 4, 5, 6, 7, 8],
                         'iterations': [200, 500, 1000, 2000],
                         'learning_rate': [0.01, 0.03, 0.05, 0.1]},
             verbose=2)

In [13]:
catb_cv_model.best_params_

{'depth': 5, 'iterations': 1000, 'learning_rate': 0.1}

In [14]:
catb_tuned = CatBoostRegressor(depth = 5, iterations = 1000, learning_rate = 0.1)
catb_tuned.fit(X_train, y_train)

0:	learn: 422.4143448	total: 9.22ms	remaining: 9.21s
1:	learn: 404.1864276	total: 13.3ms	remaining: 6.65s
2:	learn: 386.3231718	total: 17.5ms	remaining: 5.82s
3:	learn: 370.5548032	total: 21.6ms	remaining: 5.38s
4:	learn: 354.9242038	total: 26ms	remaining: 5.17s
5:	learn: 342.3403984	total: 29.6ms	remaining: 4.91s
6:	learn: 328.2370070	total: 32.8ms	remaining: 4.65s
7:	learn: 317.5056526	total: 35.1ms	remaining: 4.35s
8:	learn: 306.6243511	total: 37.1ms	remaining: 4.08s
9:	learn: 297.3147023	total: 39.1ms	remaining: 3.88s
10:	learn: 288.3685892	total: 41.2ms	remaining: 3.71s
11:	learn: 281.0996220	total: 43.1ms	remaining: 3.55s
12:	learn: 273.2254898	total: 45.2ms	remaining: 3.43s
13:	learn: 266.9003385	total: 47.2ms	remaining: 3.32s
14:	learn: 261.9092500	total: 49.2ms	remaining: 3.23s
15:	learn: 256.2637350	total: 51.1ms	remaining: 3.14s
16:	learn: 250.3667935	total: 53.2ms	remaining: 3.07s
17:	learn: 244.8631098	total: 55.2ms	remaining: 3.01s
18:	learn: 240.1540669	total: 57.3ms	rem

In [16]:
y_pred = catb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

356.665762904938